In [18]:
import os
import sys
import random
import pickle
from tqdm import tqdm

import cv2
import glob, cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import pycocotools.mask as mask

import shutil
from shutil import copyfile
import time
import errno

In [2]:
dino_cpkt_path = '/workspace/traffic_light/mmdetection/outputs_result/36_epoch_convnext2.pkl'
test_image_path = "/workspace/traffic_light/data/segmentation_coco/images/test"

In [3]:
categories=[
    'Car_VehLane',
    'Car_VehLane_IncatLft',
    'Car_VehLane_IncatRht',
    'Car_VehLane_HazLit',
    'Car_VehLane_Brake',
    'Car_VehLane_Brake_IncatLft',
    'Car_VehLane_Brake_IncatRht',
    'Car_VehLane_Brake_HazLit',
    'Car_OutgoLane',
    'Car_OutgoLane_IncatLft',
    'Car_OutgoLane_IncatRht',
    'Car_OutgoLane_HazLit',
    'Car_OutgoLane_Brake',
    'Car_OutgoLane_Brake_IncatLft',
    'Car_OutgoLane_Brake_IncatRht',
    'Car_OutgoLane_Brake_HazLit',
    'Car_IncomLane',
    'Car_IncomLane_IncatLft',
    'Car_IncomLane_IncatRht',
    'Car_IncomLane_HazLit',
    'Car_IncomLane_Brake',
    'Car_IncomLane_Brake_IncatLft',
    'Car_IncomLane_Brake_IncatRht',
    'Car_IncomLane_Brake_HazLit',
    'Car_Jun',
    'Car_Jun_IncatLft',
    'Car_Jun_IncatRht',
    'Car_Jun_HazLit',
    'Car_Jun_Brake',
    'Car_Jun_Brake_IncatLft',
    'Car_Jun_Brake_IncatRht',
    'Car_Jun_Brake_HazLit',
    'Car_Parking',
    'Car_Parking_IncatLft',
    'Car_Parking_IncatRht',
    'Car_Parking_HazLit',
    'Car_Parking_Brake',
    'Car_Parking_Brake_IncatLft',
    'Car_Parking_Brake_IncatRht',
    'Car_Parking_Brake_HazLit',
    'Bus_VehLane',
    'Bus_VehLane_IncatLft',
    'Bus_VehLane_IncatRht',
    'Bus_VehLane_HazLit',
    'Bus_VehLane_Brake',
    'Bus_VehLane_Brake_IncatLft',
    'Bus_VehLane_Brake_IncatRht',
    'Bus_VehLane_Brake_HazLit',
    'Bus_OutgoLane',
    'Bus_OutgoLane_IncatLft',
    'Bus_OutgoLane_IncatRht',
    'Bus_OutgoLane_HazLit',
    'Bus_OutgoLane_Brake',
    'Bus_OutgoLane_Brake_IncatLft',
    'Bus_OutgoLane_Brake_IncatRht',
    'Bus_OutgoLane_Brake_HazLit',
    'Bus_IncomLane',
    'Bus_IncomLane_IncatLft',
    'Bus_IncomLane_IncatRht',
    'Bus_IncomLane_HazLit',
    'Bus_IncomLane_Brake',
    'Bus_IncomLane_Brake_IncatLft',
    'Bus_IncomLane_Brake_IncatRht',
    'Bus_IncomLane_Brake_HazLit',
    'Bus_Jun',
    'Bus_Jun_IncatLft',
    'Bus_Jun_IncatRht',
    'Bus_Jun_HazLit',
    'Bus_Jun_Brake',
    'Bus_Jun_Brake_IncatRht',
    'Bus_Jun_Brake_IncatLft',
    'Bus_Jun_Brake_HazLit',
    'Bus_Parking',
    'Bus_Parking_IncatLft',
    'Bus_Parking_IncatRht',
    'Bus_Parking_HazLit',
    'Bus_Parking_Brake',
    'Bus_Parking_Brake_IncatLft',
    'Bus_Parking_Brake_IncatRht',
    'Bus_Parking_Brake_HazLit'
]

In [4]:
def category2class(labels:list):
    cls=[]
    loc=[]
    actions=[]
    for label in labels:
        action=[0,0,0,0]
        cls.append(label//40)
        loc.append(label%40//8)
        action[0]=label%40%8//4
        action[label%40%8%4]=1
        actions.append(action)
    return cls, loc, actions

In [5]:
# def rle2mask(compactRLESegmentations):
#     segmentations=[]
#     for compactRLESegmentation in compactRLESegmentations:
#         maskedArr = mask.decode(compactRLESegmentation)
#         binary_mask = maskedArr.astype('int')
#         segmentations.append(binary_mask)
#     return segmentations

In [6]:
def rle2polygon(compactRLESegmentations):
    segmentations=[]
    for compactRLESegmentation in compactRLESegmentations:
        maskedArr = mask.decode(compactRLESegmentation)
        # area = float((maskedArr > 0.0).sum())
        # adapted from https://github.com/hazirbas/coco-json-converter/blob/master/generate_coco_json.py
        contours, _ = cv2.findContours(maskedArr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        segmentation = []
        valid_poly = 0
        for contour in contours:
        # Valid polygons have >= 6 coordinates (3 points)
            if contour.size >= 6:
                segmentation.append(contour.astype(float).flatten().tolist())
                valid_poly += 1
        if valid_poly == 0:
            segmentation.append([-1]) # -1 means no poly
            # raise ValueError
        segmentations.extend(segmentation)
        # areas.append(area)
        
    return segmentations

In [7]:
def rle2polygon_mask(compactRLESegmentations):
    segmentations=[]
    masks=[]
    for compactRLESegmentation in compactRLESegmentations:
        maskedArr = mask.decode(compactRLESegmentation)
        binary_mask = maskedArr.astype('int')
        # area = float((maskedArr > 0.0).sum())
        # adapted from https://github.com/hazirbas/coco-json-converter/blob/master/generate_coco_json.py
        contours, _ = cv2.findContours(maskedArr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        segmentation = []
        valid_poly = 0
        for contour in contours:
        # Valid polygons have >= 6 coordinates (3 points)
            if contour.size >= 6:
                segmentation.append(contour.astype(float).flatten().tolist())
                masks.append(binary_mask)
                valid_poly += 1
        if valid_poly == 0:
            segmentation.append([-1]) # -1 means no poly
            masks.append([[-1]])
            # raise ValueError
        segmentations.extend(segmentation)
        # areas.append(area)
        
    return segmentations, masks #, area


In [8]:
def xyxy2xyxyn(polygons,w,h):
    return [ float(e/w) if i%2==0 else float(e/h) for polygon in polygons for i, e in enumerate(polygon)]


In [9]:
# --------------Class Param------------
agent_classes = ["Car", "Bus"]
loc_classes = ["VehLane", "OutgoLane", "IncomLane", "Jun", "Parking"]
action_classes = ["Brake", "IncatLft", "IncatRht", "HazLit"]
class_nums = [len(agent_classes), len(loc_classes), len(action_classes)]
# --------------Class Param------------

In [10]:
icons = {}

for actions in action_classes:
    target = "../../ultra/Icons/" + actions + ".png"
    icon_img = cv2.imread(target)
    icon_img = cv2.cvtColor(icon_img, cv2.COLOR_BGR2RGB)
    icons[actions] = icon_img

for actions in loc_classes:
    # print(actions)
    target = "../../ultra/Icons/" + actions + ".png"
    icon_img = cv2.imread(target)
    icon_img = cv2.cvtColor(icon_img, cv2.COLOR_BGR2RGB)
    icons[actions] = icon_img

In [11]:
def seg_plot_one_box(x, idx, img, mask, cls, loc, action, color=None, label=None, track_id=None, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness or 2  # line thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    
    c1, c2 = (
        (np.clip(int(x[0]), 0, img.shape[1]), np.clip(int(x[1]), 0, img.shape[0])),
        (np.clip(int(x[2]), 0, img.shape[1]), np.clip(int(x[3]), 0, img.shape[0])),
    )

    cv2.rectangle(img, c1, c2, color, thickness=1)

    agent_list = ["Car", "Bus"]
    loc_list = ["VehLane", "OutgoLane", "IncomLane", "Jun", "Parking"]
    action_list = ["Brake", "IncatLft", "IncatRht", "HazLit"]

    num_icon = np.sum(action)

    icon_size = int(np.min([(c2[0] - c1[0]) / num_icon, (x[3] - x[1]) / 2, 64]))
    c3 = c1[0]  # +(c2[0]-c1[0])//2-icon_size*num_icon//2

    try:
        offset_icon = 0
        for ii in range(len(action)):
            if action[ii] == 1:
                img[c1[1] : c1[1] + icon_size, c3 + offset_icon : c3 + offset_icon + icon_size, :] = (
                    cv2.resize(icons[action_list[ii]], (icon_size, icon_size), interpolation=cv2.INTER_NEAREST) * 0.5
                    + img[c1[1] : c1[1] + icon_size, c3 + offset_icon : c3 + offset_icon + icon_size, :] * 0.5
                )
                offset_icon += icon_size

        img[c2[1] - icon_size : c2[1], c3 : c3 + icon_size, :] = (
            cv2.resize(icons[loc_list[loc]], (icon_size, icon_size)) * 0.5
            + img[c2[1] - icon_size : c2[1], c3 : c3 + icon_size, :] * 0.5
        )

    except:
        pass
    
    # Expand mask dimensions to match the image
    mask = mask[c1[1] : c2[1], c1[0] : c2[0]]
    mask = mask > 0.5

    img[c1[1] : c2[1], c1[0] : c2[0], :][mask] = (
        img[c1[1] : c2[1], c1[0] : c2[0], :][mask] * 0.65 + np.array(color) * 0.35
    )


In [12]:
import shutil
import time

try:
    shutil.rmtree("/workspace/traffic_light/ultra/Result/segment/predictions/v2/img")
    print("Removed New dir")
except:
    print("Making New dir")

filepath = "/workspace/traffic_light/ultra/Result/segment/predictions/v2/img"
if not os.path.exists(filepath):
    try:
        os.makedirs(filepath)
    except OSError as exc:  # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

COLORS = [
    [255, 0, 0],  # Red
    [0, 255, 0],  # Green
    [0, 0, 255],  # Blue
    [255, 255, 0],  # Yellow
    [255, 0, 255],  # Magenta
    [0, 255, 255],  # Cyan
    [128, 0, 0],  # Maroon
    [0, 128, 0],  # Green (dark)
    [0, 0, 128],  # Navy
    [128, 128, 0],  # Olive
    [128, 0, 128],  # Purple
    [0, 128, 128],  # Teal
    [255, 165, 0],  # Orange
    [210, 180, 140],  # Tan
    [255, 192, 203],  # Pink
    [0, 128, 128],  # Teal
    [255, 99, 71],  # Tomato
    [139, 69, 19],  # Saddle Brown
    [0, 128, 0],  # Green (dark)
    [255, 20, 147],  # Deep Pink
]

Removed New dir


In [13]:
def nms(bounding_boxes, confidence_score, labels, locations, actions, polygons, threshold):
    """
    ref: https://github.com/amusi/Non-Maximum-Suppression/blob/master/nms.py
    the boxes format is xyxy
    e.g) 
    bounding_boxes = [(187, 82, 337, 317), (150, 67, 305, 282), (246, 121, 368, 304)]
    confidence_score = [0.9, 0.75, 0.8]
    threshold = 0.4
    """
    # If no bounding boxes, return empty list
    if len(bounding_boxes) == 0:
        return [], []

    # Bounding boxes
    boxes = np.array(bounding_boxes)

    # coordinates of bounding boxes
    start_x = boxes[:, 0]
    start_y = boxes[:, 1]
    end_x = boxes[:, 2]
    end_y = boxes[:, 3]

    # Confidence scores of bounding boxes
    score = np.array(confidence_score)
    
    # print(boxes.shape[0])
    # print(score.shape[0])
    
    # variables = [boxes.shape[0], score.shape[0], len(labels), len(locations), len(actions), len(polygons)]
    # reference = variables[0]
    # different_values = [value for value in variables if value != reference]

    # if different_values:
    #     print("Different values found:", different_values)
    # else:
    #     print("All values are the same.")
        
    # Picked bounding boxes
    picked_boxes = []
    picked_score = []
    picked_labels = []
    picked_locations=[]
    picked_actions=[]
    picked_polygons = []

    # Compute areas of bounding boxes
    areas = (end_x - start_x + 1) * (end_y - start_y + 1)

    # Sort by confidence score of bounding boxes
    order = np.argsort(score)

    # Iterate bounding boxes
    while order.size > 0:
        # The index of largest confidence score
        index = order[-1]

        # Pick the bounding box with largest confidence score
        picked_boxes.append(bounding_boxes[index])
        picked_score.append(confidence_score[index])
        picked_labels.append(labels[index])
        picked_locations.append(locations[index])
        picked_actions.append(actions[index])
        picked_polygons.append(polygons[index])

        # Compute ordinates of intersection-over-union(IOU)
        x1 = np.maximum(start_x[index], start_x[order[:-1]])
        x2 = np.minimum(end_x[index], end_x[order[:-1]])
        y1 = np.maximum(start_y[index], start_y[order[:-1]])
        y2 = np.minimum(end_y[index], end_y[order[:-1]])

        # Compute areas of intersection-over-union
        w = np.maximum(0.0, x2 - x1 + 1)
        h = np.maximum(0.0, y2 - y1 + 1)
        intersection = w * h

        # Compute the ratio between intersection and union
        ratio = intersection / (areas[index] + areas[order[:-1]] - intersection)

        left = np.where(ratio < threshold)
        order = order[left]
    
    # if not all(picked_labels, picked_locations, picked_actions, picked_polygons):
    #     return (False, False, False, False, False, False)
    return picked_boxes, picked_score, picked_labels, picked_locations, picked_actions, picked_polygons

# visualization
- ref : https://huggingface.co/docs/transformers/tasks/object_detection

In [17]:
nms_iou_threshold=0.6
inference_img=False
inference_txt=True

input_base_dir = '/workspace/traffic_light/data/segmentation_coco/images/test'
# target_folder = "/workspace/traffic_light/data/segmentation/test/IllegalParking01_Rain/img"  # Target Dir

output_base_dir="/workspace/traffic_light/seg_output3"

if len(glob.glob(os.path.join(input_base_dir, "*.png"))) < 100:
    print("Error : Directory Invalid")
    sys.exit(0)

cnt=1

empty_files=[]
empty_files_cnt=0
with open(dino_cpkt_path, 'rb') as f:
    datas = pickle.load(f)
    for index, data in enumerate(tqdm(datas)):
        _name=os.path.splitext(os.path.basename(data['img_path']))[0]

        # Extract city name and file name
        splited_name=_name.split('_')
        city_name = '_'.join(splited_name[:2])
        num_name = splited_name[2]
        if city_name =='Sejong_L1':
            city_name = 'Sejong_L1_00'
            num_name=splited_name[3]
        # print(city_name, type(city_name))
        # if city_name!="Sejong_L1_00":
        #     continue
        # Define the output text file path
        output_dir = os.path.join(output_base_dir, city_name, 'txt')
        output_file = os.path.join(output_dir, f"{num_name}.txt")
        
        # Create the output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)
 
        # if not os.path.exists(output_file):
        #     with open(output_file, 'w') as f3:
        #         pass
        # continue
        classes, locations, actions=category2class(data['pred_instances']['labels'].tolist())

        if not classes:
            # print('empty file: ', _name)
            with open(output_file, "w") as f2:
                pass
            empty_files_cnt+=1
            empty_files.append(_name)
            continue
        
        xyxy = data['pred_instances']['bboxes']
        scores=data['pred_instances']['scores']
        boxes=data['pred_instances']['bboxes'].numpy()
        ori_h, ori_w=datas[0]['ori_shape']

        compactRLESegmentation = data['pred_instances']['masks']
        # masks = rle2mask(compactRLESegmentation)
        if inference_img:
            polygons, masks = rle2polygon_mask(compactRLESegmentation)
        else:
            polygons=rle2polygon(compactRLESegmentation)

        # apply NMS
        if nms_iou_threshold != 1:
            _, scores, classes, locations, actions, polygons=nms(boxes, scores, classes, locations, actions, polygons, nms_iou_threshold)
        
        if inference_img:
            target_img = np.array(Image.open(data['img_path']).convert("RGB"))
            for i in range(len(scores)):
                if masks[i][0][0] == -1 or (not classes):
                    continue
                seg_plot_one_box(
                    xyxy[i],
                    i,
                    target_img,
                    np.array(masks[i]),
                    classes[i],
                    locations[i],
                    actions[i],
                    color=COLORS[i % len(COLORS)],
                )

            target_img = target_img.copy()

            path = "/workspace/traffic_light/ultra/Result/segment/predictions/v2/img/" + _name + ".png"
            cv2.imwrite(path, target_img[:, :, ::-1])
        # else:
            # target_img = np.array(Image.open(data['img_path']).convert("RGB"))
        #     for i in range(len(picked_score)):
        #         if masks[i][0][0] == -1 or (not picked_labels):
        #             continue
        #         seg_plot_one_box(
        #             xyxy[i],
        #             i,
        #             target_img,
        #             np.array(masks[i]),
        #             picked_labels[i],
        #             picked_locations[i],
        #             picked_actions[i],
        #             color=COLORS[i % len(COLORS)],
        #         )
            
        #     print(_name)
        #     plt.rcParams["figure.figsize"] = [20, 10]
        #     plt.imshow(target_img)
            
        if inference_txt:
            with open(output_file, "w") as f:
                for score, cls, loc, action, poly in zip(scores, classes, locations, actions, polygons):
                    poly=xyxy2xyxyn(polygons, ori_w, ori_h)
                    score=score.item()
                    if (not cls) and len(poly) > 5 and poly[0] > -1 and score > 0.25:

                        result_txt = (
                            str(score) # cs
                            + " "
                            + str(cls)
                            + " "
                            + str(loc)
                            + " "
                            + " ".join(map(str, action))
                            + " "
                            + " ".join(map(str, poly)) # polygon
                            + "\n"
                        )
                        f.write(result_txt)
                    

        
        # if index >1:
        #     break
# print('cnt',cnt)
print('empty_files: ', empty_files)
print('empty_files_cnt: ',empty_files_cnt)


100%|██████████| 8909/8909 [03:34<00:00, 41.46it/s]

empty_files:  ['Banseok02_Snow_00028957', 'CounterClock_Fog_00040121', 'Banseok02_Snow_00029012', 'Ulsan01_Day_00018554', 'CounterClock_Fog_00040136', 'CounterClock_Fog_00040076', 'OiSam_Day_00020123', 'Ulsan01_Day_00018549', 'OiSam_Day_00020151', 'Banseok02_Snow_00028977', 'CounterClock_Fog_00040101', 'Banseok02_Snow_00029002', 'CounterClock_Fog_00040146', 'Banseok02_Snow_00029017', 'OiSam_Day_00020159', 'Ulsan01_Day_00018559', 'Banseok02_Snow_00028982', 'CounterClock_Fog_00040016', 'CounterClock_Fog_00040071', 'Banseok02_Snow_00028932', 'Banseok02_Snow_00025672', 'OiSam_Day_00020531', 'CounterClock_Fog_00040111', 'CounterClock_Fog_00040106', 'CounterClock_Fog_00040126', 'CounterClock_Fog_00040081', 'OiSam_Day_00020131', 'Banseok02_Snow_00028972', 'Banseok02_Snow_00028952', 'Banseok02_Snow_00028962', 'Banseok02_Snow_00028987', 'CounterClock_Fog_00040141', 'CounterClock_Fog_00040011', 'Banseok02_Snow_00028992', 'Ulsan01_Day_00018564', 'Banseok02_Snow_00028927', 'CounterClock_Fog_000401

In [90]:
for folder_name in os.listdir('/workspace/traffic_light/seg_output'):
    
    test_dir=f'/workspace/traffic_light/data/segmentation/test/{folder_name}/img'
    test_list = os.listdir(test_dir)
    out_path=f'/workspace/traffic_light/seg_output/{folder_name}/txt'
    cnt=0
    for tl in test_list:
        name=tl.split('.')[0]
        # print(name)
        if not os.path.exists(os.path.join(out_path,f'{name}.txt')):
            cnt+=1
            with open(os.path.join(out_path,f'{name}.txt'), 'w') as f3:
                pass
    print(folder_name,' ',cnt)

Doryong_Cons   0
Ulsan01_Day   0
Yongsan_Cons   0
IllegalParking04_Rain   0


FileNotFoundError: [Errno 2] No such file or directory: '/workspace/traffic_light/data/segmentation/test/Sejong_L1/img'